In [1]:
import requests
import pandas as pd
import pymysql
import numpy as np
import urllib.parse as parse

In [2]:
name_url = 'https://api.defistation.io/defiTvlList'
bnblockedList_data_url = 'https://api.defistation.io/bnblockedList/{defiName}?days=30'
tvl_data_list = 'https://api.defistation.io/chart/{defiName}?days=30'
token = "Basic OmMyNDVhNGEyLTYxZjgtMTFlYi1hZTkzLTAyNDJhYzEzMDAwMg=="

In [3]:
def get_data(url):
#     url = parse.quote(url)
    print(url)
    response = requests.get(url, headers = {"Authorization":token})
    return response.json()

In [4]:
data = get_data(name_url)

https://api.defistation.io/defiTvlList


In [5]:
defi_names = []
for defi_details in data:
    defi_names.append([defi_details['name'], defi_details['token']])
defi_names

[['pancake', 'CAKE'],
 ['Venus', 'XVS'],
 ['Ellipsis Finance', 'EPS'],
 ['MDEX', 'MDX'],
 ['Belt Finance', 'BELT'],
 ['Alpaca Finance', 'ALPACA'],
 ['Autofarm', 'AUTO'],
 ['PancakeBunny', 'BUNNY'],
 ['O3 Swap', 'O3'],
 ['Rabbit Finance', 'RABBIT'],
 ['Biswap', 'BSW'],
 ['beefy.finance', 'BIFI'],
 ['Bakery Swap', 'BAKE'],
 ['Wault.Finance', 'WEX'],
 ['ApeSwap', 'BANANA'],
 ['ForTube', 'FOR'],
 ['ACryptoS', 'ACS'],
 ['BTC Standard Hashrate Token', 'BTCST'],
 ['Nerve Finance', 'NRV'],
 ['Jetfuel.Finance', 'FUEL'],
 ['dForce', 'DF'],
 ['BunnyPark', 'BP'],
 ['Cream Finance', 'CREAM'],
 ['Linear Finance', 'LINA'],
 ['Alpha Homora', 'ALPHA'],
 ['SwampFinance', 'SWAMP'],
 ['Dopple Finance', 'DOP'],
 ['Nominex', 'NMX'],
 ['Growing Farm', 'GROW'],
 ['DeFireX', 'DFX'],
 ['ApeRocket', None],
 ['dFuture', 'DFT'],
 ['Definix', None],
 ['Fleta Connect', 'CHERRY'],
 ['bZx Protocol', 'BZRX'],
 ['Bunicorn', None],
 ['Goose Finance', 'EGG'],
 ['TreeDefi', 'TREE'],
 ['TEN', 'TENFI'],
 ['TimeWarp', None],


In [6]:
def get_data_from_dataset(data_set, group_key, date_key):
    if group_key in data_set and date_key in data_set[group_key]:
        return data_set[group_key][date_key]
    return np.nan

In [7]:
def assemble_to_rows(defiData_bnblocked, tvl_data, defi_name_details):
    rows = []
    if 'result' in tvl_data:
        for date_key in tvl_data['result']:
            tmp_defi_name_details = defi_name_details.copy()
            tmp_defi_name_details.extend([date_key,
                                          get_data_from_dataset(defiData_bnblocked, 'result', date_key), 
                                          get_data_from_dataset(defiData_bnblocked, 'price', date_key), 
                                          get_data_from_dataset(defiData_bnblocked, 'marketCap', date_key), 
                                          get_data_from_dataset(defiData_bnblocked, 'holders', date_key), 
                                          get_data_from_dataset(tvl_data, 'result', date_key)
                                         ])
            rows.append(tmp_defi_name_details)
    return rows

In [8]:
defi_data_set = []
for defi_name_details in defi_names:
    defiName = parse.quote(defi_name_details[0])
    bnblocked_data = get_data(bnblockedList_data_url.format(defiName=defiName))
    tvl_data = get_data(tvl_data_list.format(defiName=defiName))
    defi_name_data = assemble_to_rows(bnblocked_data, tvl_data, defi_name_details.copy())
    if defi_name_data:
        defi_data_set.extend(defi_name_data)
    print(defiName)

https://api.defistation.io/bnblockedList/pancake?days=30
https://api.defistation.io/chart/pancake?days=30
pancake
https://api.defistation.io/bnblockedList/Venus?days=30
https://api.defistation.io/chart/Venus?days=30
Venus
https://api.defistation.io/bnblockedList/Ellipsis%20Finance?days=30
https://api.defistation.io/chart/Ellipsis%20Finance?days=30
Ellipsis%20Finance
https://api.defistation.io/bnblockedList/MDEX?days=30
https://api.defistation.io/chart/MDEX?days=30
MDEX
https://api.defistation.io/bnblockedList/Belt%20Finance?days=30
https://api.defistation.io/chart/Belt%20Finance?days=30
Belt%20Finance
https://api.defistation.io/bnblockedList/Alpaca%20Finance?days=30
https://api.defistation.io/chart/Alpaca%20Finance?days=30
Alpaca%20Finance
https://api.defistation.io/bnblockedList/Autofarm?days=30
https://api.defistation.io/chart/Autofarm?days=30
Autofarm
https://api.defistation.io/bnblockedList/PancakeBunny?days=30
https://api.defistation.io/chart/PancakeBunny?days=30
PancakeBunny
http

Lever%20Network
https://api.defistation.io/bnblockedList/Xpool?days=30
https://api.defistation.io/chart/Xpool?days=30
Xpool
https://api.defistation.io/bnblockedList/AnySwap?days=30
https://api.defistation.io/chart/AnySwap?days=30
AnySwap
https://api.defistation.io/bnblockedList/Puppyswap?days=30
https://api.defistation.io/chart/Puppyswap?days=30
Puppyswap
https://api.defistation.io/bnblockedList/ARIES%20FINANCIAL?days=30
https://api.defistation.io/chart/ARIES%20FINANCIAL?days=30
ARIES%20FINANCIAL
https://api.defistation.io/bnblockedList/Kebab%20Finance?days=30
https://api.defistation.io/chart/Kebab%20Finance?days=30
Kebab%20Finance
https://api.defistation.io/bnblockedList/Fruits%20Adventures?days=30
https://api.defistation.io/chart/Fruits%20Adventures?days=30
Fruits%20Adventures
https://api.defistation.io/bnblockedList/TuringMachine?days=30
https://api.defistation.io/chart/TuringMachine?days=30
TuringMachine
https://api.defistation.io/bnblockedList/Waypoint?days=30
https://api.defistat

In [9]:
defi_data_set

[['pancake', 'CAKE', '1623758400', nan, nan, nan, nan, 7995127242],
 ['pancake', 'CAKE', '1623844800', nan, nan, nan, nan, 8467909290],
 ['pancake', 'CAKE', '1623931200', nan, nan, nan, nan, 9330960916],
 ['pancake', 'CAKE', '1624017600', nan, nan, nan, nan, 11242660842],
 ['pancake', 'CAKE', '1624104000', nan, nan, nan, nan, 10193326793],
 ['pancake', 'CAKE', '1624190400', nan, nan, nan, nan, 9519424676],
 ['pancake', 'CAKE', '1624276800', nan, nan, nan, nan, 8577632119],
 ['pancake', 'CAKE', '1624363200', nan, nan, nan, nan, 7613418956],
 ['pancake', 'CAKE', '1624449600', nan, nan, nan, nan, 8376260521],
 ['pancake', 'CAKE', '1624536000', nan, nan, nan, nan, 10674538572],
 ['pancake', 'CAKE', '1624622400', nan, nan, nan, nan, 6562561293],
 ['pancake', 'CAKE', '1624708800', nan, nan, nan, nan, 6134084243],
 ['pancake', 'CAKE', '1624795200', nan, nan, nan, nan, 6318083538],
 ['pancake', 'CAKE', '1624881600', nan, nan, nan, nan, 6601801884],
 ['pancake', 'CAKE', '1624968000', nan, nan, 

In [52]:
def __connect__():
    db_user = 'coin'
    db_password = 'dragonstone#123'
    db_name = 'master_data'
    db_host = 'localhost'
    return pymysql.connect(
        unix_socket=None,
        host=db_host,
        user=db_user,
        password=db_password,
        db=db_name,
        charset='latin1',
        cursorclass=pymysql.cursors.DictCursor)


def get_cursor(connection):
    if connection is None:
        connection = __connect__()
    connection.ping(reconnect=True)
    return connection.cursor()


def insert_defi_neg_div_score(df):
    df = df.astype(object).where(pd.notnull(df), None)
    connection = __connect__()
    cursor = get_cursor(connection)
    for i, row in df.iterrows():
        try:
            dt = row["date"].timestamp()
            symbol = row['token'] if row['token'] else ' '
            print(row['date'], row['name'], row['token'], row['bnb_locked'], row['price'], row['mrk_cap'], row['holders'], row['tvl'])
            sql = """
            INSERT INTO `defistation_data` (`timestamp_dt`, `name`, `token`, `price`, `bnb_locked`, `mrk_cap`, `holders`, `tvl`) VALUES (FROM_UNIXTIME(%s), %s, %s, %s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE
            `token` = VALUES(token),
            `price` = VALUES(price),
            `bnb_locked` = VALUES(bnb_locked),
            `mrk_cap` = VALUES(mrk_cap),
            `holders` = VALUES(holders),
            `tvl` = VALUES(tvl)
            """
            cursor.execute(sql, (dt , row['name'], symbol, row['bnb_locked'], row['price'], row['mrk_cap'], row['holders'], row['tvl']))
        except Exception as e:
            print(e)
    connection.commit()
    cursor.close()
    connection.close()

In [53]:
insert_defi_neg_div_score(df)

2021-04-01 00:00:00 Venus XVS 20710648.043879468 62.785053471961604 581035990.303794 0.0 6487981220
2021-04-02 00:00:00 Venus XVS 21185079.37280539 58.002299378239265 536716907.40117896 0.0 7457936024
2021-04-03 00:00:00 Venus XVS 22186491.931395605 52.18778578831958 486357919.0305561 0.0 7608857407
2021-04-04 00:00:00 Venus XVS 22601180.085801877 57.77711124734978 534872999.6998267 0.0 7599646803
2021-04-05 00:00:00 Venus XVS 23223931.097593516 55.01538253281552 511017215.757199 0.0 7915672544
2021-04-06 00:00:00 Venus XVS 20914561.85206597 57.658578678298944 539272125.7259693 0.0 7881515643
2021-04-07 00:00:00 Venus XVS 19726477.562690284 58.44 545499310.2054386 0.0 7987490441
2021-04-08 00:00:00 Venus XVS 19994292.64905405 58.56 547354671.4419367 17020.0 7984120940
2021-04-09 00:00:00 Venus XVS 18549659.248659283 63.61 593132631.263695 17020.0 7773883946
2021-04-10 00:00:00 Venus XVS 17744232.338401135 62.32 582914931.8389709 16952.0 8421678831
2021-04-11 00:00:00 Venus XVS 18225303

2021-04-25 00:00:00 Ellipsis Finance EPS 0.0 1.76 154525183.21069825 6991.0 2913636379
2021-04-26 00:00:00 Ellipsis Finance EPS 0.0 1.91 169843702.8645033 6919.0 2897834549
2021-04-27 00:00:00 Ellipsis Finance EPS 0.0 1.86 175906802.96346897 6948.0 2860276467
2021-04-28 00:00:00 Ellipsis Finance EPS 0.0 2.6 248886879.56661826 6949.0 2609099675
2021-04-29 00:00:00 Ellipsis Finance EPS 0.0 2.54 255655881.22175285 7007.0 2560859767
2021-04-30 00:00:00 Ellipsis Finance EPS 0.0 2.54 255655881.22175285 7270.0 2429152020
2021-04-01 00:00:00 MDEX MDX 0.0 None None None 0
2021-04-02 00:00:00 MDEX MDX 0.0 None None None 0
2021-04-03 00:00:00 MDEX MDX 0.0 None None None 0
2021-04-04 00:00:00 MDEX MDX 0.0 None None None 0
2021-04-05 00:00:00 MDEX MDX 0.0 None None None 0
2021-04-06 00:00:00 MDEX MDX 0.0 None None None 0
2021-04-07 00:00:00 MDEX MDX 0.0 None None None 0
2021-04-08 00:00:00 MDEX MDX 0.0 None None None 0
2021-04-09 00:00:00 MDEX MDX 0.0 None None None 0
2021-04-10 00:00:00 MDEX MDX 0

2021-04-04 00:00:00 ACryptoS ACS 158867.27382103723 64.10851776986235 35314822.65556783 0.0 320364608
2021-04-05 00:00:00 ACryptoS ACS 157544.49032311415 59.19876360128139 32831727.29209349 0.0 325455727
2021-04-06 00:00:00 ACryptoS ACS 148953.44071726655 62.49023380581577 34849645.84284671 0.0 332881815
2021-04-07 00:00:00 ACryptoS ACS 142397.121467454 61.77 34697189.17024066 0.0 331005761
2021-04-08 00:00:00 ACryptoS ACS 139800.75221684467 60.41 34098822.82430283 4285.0 333996307
2021-04-09 00:00:00 ACryptoS ACS 138945.83314350992 64.69 36712849.88870693 4285.0 323484636
2021-04-10 00:00:00 ACryptoS ACS 132034.57378692905 65.12 37139473.09247838 4284.0 345201775
2021-04-11 00:00:00 ACryptoS ACS 124084.0603093653 69.62 40051862.41605727 4280.0 333312103
2021-04-12 00:00:00 ACryptoS ACS 114416.2415525243 63.6 36630362.89785538 4263.0 353700542
2021-04-13 00:00:00 ACryptoS ACS 107012.11297587691 67.23 38959889.91504198 4245.0 334870600
2021-04-14 00:00:00 ACryptoS ACS 108096.25490544806

2021-04-29 00:00:00 ForTube FOR 97162.990462 0.123331 69910853.85724856 0.0 278215781
2021-04-30 00:00:00 ForTube FOR 98612.790392 0.123331 69910853.85724856 0.0 288140036
2021-04-01 00:00:00 ApeSwap BANANA 0.0 0.0 0.0 0.0 36106647
2021-04-02 00:00:00 ApeSwap BANANA 0.0 0.0 0.0 0.0 34648182
2021-04-03 00:00:00 ApeSwap BANANA 0.0 0.0 0.0 0.0 33543789
2021-04-04 00:00:00 ApeSwap BANANA 0.0 0.0 0.0 0.0 31512082
2021-04-05 00:00:00 ApeSwap BANANA 0.0 0.0 0.0 0.0 33455239
2021-04-06 00:00:00 ApeSwap BANANA 0.0 0.0 0.0 0.0 38690345
2021-04-07 00:00:00 ApeSwap BANANA 0.0 0.0 0.0 0.0 34113150
2021-04-08 00:00:00 ApeSwap BANANA 0.0 0.0 0.0 0.0 40080615
2021-04-09 00:00:00 ApeSwap BANANA 0.0 0.0 0.0 0.0 40661040
2021-04-10 00:00:00 ApeSwap BANANA 0.0 0.0 0.0 0.0 42483146
2021-04-11 00:00:00 ApeSwap BANANA 0.0 0.0 0.0 0.0 42561668
2021-04-12 00:00:00 ApeSwap BANANA 0.0 0.0 0.0 0.0 44877032
2021-04-13 00:00:00 ApeSwap BANANA 0.0 0.0 0.0 0.0 45005967
2021-04-14 00:00:00 ApeSwap BANANA 0.0 0.0 0.0 0

2021-04-06 00:00:00 Goose Finance EGG 0.0 29.892160308213548 41323084.25763853 0.0 116563263
2021-04-07 00:00:00 Goose Finance EGG 0.0 27.41 38676707.29301909 0.0 126237963
2021-04-08 00:00:00 Goose Finance EGG 0.0 26.22 37319962.51589776 11463.0 128743155
2021-04-09 00:00:00 Goose Finance EGG 0.0 26.17 38979755.63915933 11463.0 130484639
2021-04-10 00:00:00 Goose Finance EGG 0.0 22.96 33922910.41449585 11413.0 135025609
2021-04-11 00:00:00 Goose Finance EGG 0.0 24.1 36215350.22957443 11348.0 128670342
2021-04-12 00:00:00 Goose Finance EGG 0.0 22.39 34052324.9398652 11327.0 133655475
2021-04-13 00:00:00 Goose Finance EGG 0.0 22.0 34230231.73735099 11360.0 127107568
2021-04-14 00:00:00 Goose Finance EGG 0.0 21.65 34087459.67620656 11319.0 129241077
2021-04-15 00:00:00 Goose Finance EGG 0.0 22.12 35107077.16728614 11263.0 124955070
2021-04-16 00:00:00 Goose Finance EGG 0.0 17.29 27762105.58328097 10023.0 124955070
2021-04-17 00:00:00 Goose Finance EGG 0.0 15.01 24432976.86367491 9105.0 1

2021-04-23 00:00:00 JulSwap JULD 57615.73800741585 0.097739 37148167.11232635 95460.0 26967868
2021-04-24 00:00:00 JulSwap JULD 56574.72557336407 0.093794 35507604.90591439 95753.0 29020028
2021-04-25 00:00:00 JulSwap JULD 54788.455194087386 0.096503 36790771.03595666 95709.0 26675639
2021-04-26 00:00:00 JulSwap JULD 51558.50197971583 0.101648 38657934.48929472 95820.0 27777369
2021-04-27 00:00:00 JulSwap JULD 51032.3894196569 0.124 46770268.28534872 96247.0 27662534
2021-04-28 00:00:00 JulSwap JULD 48941.19366835326 0.156938 59341779.95761234 96915.0 27366203
2021-04-29 00:00:00 JulSwap JULD 53341.708225418566 0.175049 66454922.00699974 97455.0 30427882
2021-04-30 00:00:00 JulSwap JULD 52164.47261190664 0.175049 66454922.00699974 99055.0 32188705
2021-04-01 00:00:00 ACoconut AC 0.0 None None 0.0 0
2021-04-02 00:00:00 ACoconut AC 0.0 None None 0.0 0
2021-04-03 00:00:00 ACoconut AC 0.0 None None 0.0 0
2021-04-04 00:00:00 ACoconut AC 0.0 None None 0.0 0
2021-04-05 00:00:00 ACoconut AC 0.

2021-04-18 00:00:00 BiFi BIFI 6121.12086219276 0.090114 9019916.03783962 0.0 10647960
2021-04-19 00:00:00 BiFi BIFI 6228.3363223835595 0.084742 8482415.00950343 0.0 11038291
2021-04-20 00:00:00 BiFi BIFI 5675.965970493476 0.099377 9955611.18014584 0.0 10005446
2021-04-21 00:00:00 BiFi BIFI 5628.649709640865 0.103253 10386979.1397791 0.0 11147372
2021-04-22 00:00:00 BiFi BIFI 5865.898380870916 0.056243 5680599.55357978 0.0 11255466
2021-04-23 00:00:00 BiFi BIFI 6388.092982483064 0.066194 6686169.63961165 0.0 10007435
2021-04-24 00:00:00 BiFi BIFI 6312.2883183492795 0.06139 6203260.47817994 0.0 10327279
2021-04-25 00:00:00 BiFi BIFI 6563.7687055157185 0.079749 8071950.67771664 0.0 10008650
2021-04-26 00:00:00 BiFi BIFI 6386.466301661456 0.085669 8675946.623314 0.0 10785592
2021-04-27 00:00:00 BiFi BIFI 6434.6437663587385 0.079613 8099113.02103854 0.0 11048350
2021-04-28 00:00:00 BiFi BIFI 5954.699598736723 0.080166 8165712.28622747 0.0 12031189
2021-04-29 00:00:00 BiFi BIFI 6134.99029169

2021-04-13 00:00:00 Demex SWTH 79.7039693681355 0.07664 108147394.714769 1565.0 4395856
2021-04-14 00:00:00 Demex SWTH 77.79407963309488 0.073149 103533599.89292671 1551.0 4570925
2021-04-15 00:00:00 Demex SWTH 66.45202698108213 0.072524 102750517.84334469 1565.0 4491747
2021-04-16 00:00:00 Demex SWTH 66.72302698108213 0.077923 110506899.07660405 1570.0 4516423
2021-04-17 00:00:00 Demex SWTH 66.72302698108213 0.073378 103176082.26244178 1571.0 4677448
2021-04-18 00:00:00 Demex SWTH 66.81016859341207 0.082997 115993709.94077699 1576.0 4413744
2021-04-19 00:00:00 Demex SWTH 66.59816859341207 0.072738 102558289.305166 1576.0 4592978
2021-04-20 00:00:00 Demex SWTH 70.27560885046603 0.080247 112393626.0783816 1579.0 4270748
2021-04-21 00:00:00 Demex SWTH 64.6536070223821 0.074405 104362193.58450034 0.0 4393958
2021-04-22 00:00:00 Demex SWTH 64.72968677545838 0.061555 86194962.50880559 1588.0 4197400
2021-04-23 00:00:00 Demex SWTH 67.11777925336038 0.0663 93113187.44976206 1628.0 3889959
202

2021-04-07 00:00:00 ARIES FINANCIAL AFI 0.0 1.91 0.0 0.0 0
2021-04-08 00:00:00 ARIES FINANCIAL AFI 6.309259479998 1.99 0.0 15.0 1093
2021-04-09 00:00:00 ARIES FINANCIAL AFI 6.309259479998 1.98 0.0 15.0 1093
2021-04-10 00:00:00 ARIES FINANCIAL AFI 6.309259479998 1.81 0.0 15.0 1093
2021-04-11 00:00:00 ARIES FINANCIAL AFI 6.309259479998 1.83 0.0 15.0 1093
2021-04-12 00:00:00 ARIES FINANCIAL AFI 6.309259479998 1.83 0.0 15.0 1103
2021-04-13 00:00:00 ARIES FINANCIAL AFI 4.649237539056 1.83 0.0 15.0 1103
2021-04-14 00:00:00 ARIES FINANCIAL AFI 4.649237539056 1.85 0.0 15.0 1103
2021-04-15 00:00:00 ARIES FINANCIAL AFI 4.649237539056 1.92 0.0 15.0 1103
2021-04-16 00:00:00 ARIES FINANCIAL AFI 4.649237539056 1.91 0.0 17.0 1103
2021-04-17 00:00:00 ARIES FINANCIAL AFI 4.649237539056 1.75 0.0 17.0 1103
2021-04-18 00:00:00 ARIES FINANCIAL AFI 302.194353813076 1.66 0.0 17.0 9525
2021-04-19 00:00:00 ARIES FINANCIAL AFI 324.056331095615 1.66 0.0 17.0 9541
2021-04-20 00:00:00 ARIES FINANCIAL AFI 205.24941

2021-04-29 00:00:00 BlackHoleSwap None 0.0 None None None 181700
2021-04-30 00:00:00 BlackHoleSwap None 0.0 None None None 181700
2021-04-01 00:00:00 Pumpy None 0.0 None None None 0
2021-04-02 00:00:00 Pumpy None 0.0 None None None 0
2021-04-03 00:00:00 Pumpy None 0.0 None None None 0
2021-04-04 00:00:00 Pumpy None 0.0 None None None 0
2021-04-05 00:00:00 Pumpy None 0.0 None None None 0
2021-04-06 00:00:00 Pumpy None 0.0 None None None 0
2021-04-07 00:00:00 Pumpy None 0.0 None None None 0
2021-04-08 00:00:00 Pumpy None 0.0 None None None 0
2021-04-09 00:00:00 Pumpy None 0.0 None None None 0
2021-04-10 00:00:00 Pumpy None 0.0 None None None 0
2021-04-11 00:00:00 Pumpy None 0.0 None None None 0
2021-04-12 00:00:00 Pumpy None 0.0 None None None 0
2021-04-13 00:00:00 Pumpy None 0.0 None None None 0
2021-04-14 00:00:00 Pumpy None 0.0 None None None 0
2021-04-15 00:00:00 Pumpy None 0.0 None None None 0
2021-04-16 00:00:00 Pumpy None 0.0 None None None 0
2021-04-17 00:00:00 Pumpy None 0.0 Non

In [10]:
df = pd.DataFrame(defi_data_set, columns=['name', 'token','date','bnb_locked','price','mrk_cap','holders','tvl'])
df['date'] = pd.to_datetime(pd.to_datetime(df['date'], unit='s').dt.strftime('%Y-%m-%d'))
df.to_csv('defistation_data.csv')
df

,name,token,date,bnb_locked,price,mrk_cap,holders,tvl
0,pancake,CAKE,2021-06-15,NaN,NaN,NaN,NaN,7995127242
1,pancake,CAKE,2021-06-16,NaN,NaN,NaN,NaN,8467909290
2,pancake,CAKE,2021-06-17,NaN,NaN,NaN,NaN,9330960916
3,pancake,CAKE,2021-06-18,NaN,NaN,NaN,NaN,11242660842
4,pancake,CAKE,2021-06-19,NaN,NaN,NaN,NaN,10193326793
...,...,...,...,...,...,...,...,...
2275,EmiSwap,None,2021-07-10,NaN,NaN,NaN,NaN,0
2276,EmiSwap,None,2021-07-11,NaN,NaN,NaN,NaN,0
2277,EmiSwap,None,2021-07-12,NaN,NaN,NaN,NaN,0
2278,EmiSwap,None,2021-07-13,NaN,NaN,NaN,NaN,0


In [11]:
len(df.token.unique())

60

In [13]:
df.date.max()

Timestamp('2021-07-14 00:00:00')